In [1]:
import os
import urllib

import numpy as np
import tensorflow as tf

In [4]:
# Download dataset 
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

if not os.path.exists(IRIS_TRAINING):
    raw = urllib.urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "w") as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urllib.urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "w") as f:
        f.write(raw)

In [5]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)


In [8]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]
feature_columns

[_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None)]

In [9]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
clf = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns, 
                                    hidden_units=[10,20,10],
                                    n_classes=3,
                                    model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6c7aa20f90>, '_model_dir': '/tmp/iris_model', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}


In [10]:
# Training input
def get_train_inputs():
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)
    
    return x, y

In [17]:
# Fit model
clf.fit(input_fn=get_train_inputs, steps=2000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-4000
INFO:tensorflow:Saving checkpoints for 4001 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:loss = 0.0354929, step = 4001
INFO:tensorflow:global_step/sec: 563.051
INFO:tensorflow:loss = 0.0351204, step = 4101 (0.178 sec)
INFO:tensorflow:global_step/sec: 601.677
INFO:tensorflow:loss = 0.0348026, step = 4201 (0.166 sec)
INFO:tensorflow:global_step/sec: 539.482
INFO:tensorflow:loss = 0.0344666, step = 4301 (0.185 sec)
INFO:tensorflow:global_step/sec: 593.578
INFO:tensorflow:loss = 0.03414, step = 4401 (0.168 sec)
INFO:tensorflow:global_step/sec: 598

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7f6c7aa20ed0>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'embedding_lr_multipliers': None, 'optimizer': None, 'dropout': None, 'gradient_clip_norm': None, 'activation_fn': <function relu at 0x7f6c88fd3140>, 'input_layer_min_slice_size': None})

In [18]:
# Define the test inputs
def get_test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)

    return x, y


In [19]:
# Evaluate accuracy
accuracy_score = clf.evaluate(input_fn=get_test_inputs, steps=1)
accuracy_score

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-07-24-15:29:14
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-6000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-07-24-15:29:14
INFO:tensorflow:Saving dict for global step 6000: accuracy = 0.966667, global_step = 6000, loss = 0.106229


{'accuracy': 0.96666664, 'global_step': 6000, 'loss': 0.10622904}

In [22]:
# Classify two new flower samples.
def new_samples():
    return np.array(
        [[6.4, 3.2, 4.5, 1.5],
        [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

predictions = list(clf.predict(input_fn=new_samples))

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predictions))

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-6000
New Samples, Class Predictions:    [1, 2]

